In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, mean_squared_error, mean_absolute_error, r2_score
from nltk.tokenize import word_tokenize
import nltk
import torch
from torch.nn.utils.rnn import pad_sequence

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Kiaver\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
df_gen = pd.read_csv("../../data/generated/gen_solutions.csv")
df_gen = df_gen[["spec", "solution"]]
df_gen.columns = ["task", "solution"]
df_gen["target"] = 1.0
df_gen

,task,solution,target
0,c0df7d49-26f5-451c-b44a-1e0bca60bca5,\r\nn = 1\r\nwhile n:\r\n n = int(input())\...,1.0
1,c0df7d49-26f5-451c-b44a-1e0bca60bca5,\r\ncount = 0\r\nnum = input()\r\nwhile num !=...,1.0
2,c0df7d49-26f5-451c-b44a-1e0bca60bca5,sequence = []\r\nwhile True:\r\n num = int(...,1.0
3,4e5b21c0-e86f-4eac-82b6-1a0d00ae4199,a=[]\r\nwhile True:\r\n b=input()\r\n if b...,1.0
4,4e5b21c0-e86f-4eac-82b6-1a0d00ae4199,\r\nmax_val = float('-inf')\r\ncount = 0\r\n\r...,1.0
...,...,...,...
3376,c4ced8e6-dec6-4f4a-8a79-f5800795a08b,\r\nn = int(input())\r\nmin_colored = 0\r\nmax...,1.0
3377,c4ced8e6-dec6-4f4a-8a79-f5800795a08b,\r\nn = int(input())\r\nmin_cubes = float('inf...,1.0
3378,df2fe73c-bfa4-40e5-b77c-d2ad076197ca,"\r\ndef can_distribute(max_time, n, k, s, cand...",1.0
3379,df2fe73c-bfa4-40e5-b77c-d2ad076197ca,"\r\ndef min_time_to_eat_candies(n, k, s, bags)...",1.0


In [3]:
df_real = pd.read_csv("../../data/db_attempts.csv")
df_real = df_real[["task", "programText"]]
df_real.columns = ["task", "solution"]
df_real["target"] = 0.0 
df_real

,task,solution,target
0,4659a273-9c89-4ff8-a1bd-6d0ae87e9915,x=int (input())\r\nx=x-2\r\nif x%10==1 and x!=...,0.0
1,84118453-37c4-4256-a8f0-5dc2707d4e35,a=str(bin(int(input())))\r\nprint(a[2:]),0.0
2,b8c265d0-999d-49e2-a5fd-eed95708bf97,n = int(input())\r\n\r\ngr = [[] * n for i in ...,0.0
3,84118453-37c4-4256-a8f0-5dc2707d4e35,a=str(bin(int(input())))\r\nprint(a[2:]),0.0
4,84118453-37c4-4256-a8f0-5dc2707d4e35,a=str(bin(int(input())))\r\nprint(a[2:]),0.0
...,...,...,...
5946,49c03922-c57e-464a-8191-9ebef2615808,"a, b, c = map(int, input().split())\nif c > a ...",0.0
5947,2173e6e8-cca6-47c7-8ef6-ea93d3e736f4,"n,*a=map(int,open(0).read().split())\na=sorted...",0.0
5948,2a42d873-b1fe-4256-a488-91db4eaa8d9f,n=list(input())\nd=sorted(list(input()))\nif m...,0.0
5949,48f237db-e593-4be6-b3f9-10757a1d07ec,a=int(input())\nprint(a),0.0


In [4]:
df_unite = pd.concat([df_gen, df_real], axis=0, ignore_index=True)
df_unite

,task,solution,target
0,c0df7d49-26f5-451c-b44a-1e0bca60bca5,\r\nn = 1\r\nwhile n:\r\n n = int(input())\...,1.0
1,c0df7d49-26f5-451c-b44a-1e0bca60bca5,\r\ncount = 0\r\nnum = input()\r\nwhile num !=...,1.0
2,c0df7d49-26f5-451c-b44a-1e0bca60bca5,sequence = []\r\nwhile True:\r\n num = int(...,1.0
3,4e5b21c0-e86f-4eac-82b6-1a0d00ae4199,a=[]\r\nwhile True:\r\n b=input()\r\n if b...,1.0
4,4e5b21c0-e86f-4eac-82b6-1a0d00ae4199,\r\nmax_val = float('-inf')\r\ncount = 0\r\n\r...,1.0
...,...,...,...
9327,49c03922-c57e-464a-8191-9ebef2615808,"a, b, c = map(int, input().split())\nif c > a ...",0.0
9328,2173e6e8-cca6-47c7-8ef6-ea93d3e736f4,"n,*a=map(int,open(0).read().split())\na=sorted...",0.0
9329,2a42d873-b1fe-4256-a488-91db4eaa8d9f,n=list(input())\nd=sorted(list(input()))\nif m...,0.0
9330,48f237db-e593-4be6-b3f9-10757a1d07ec,a=int(input())\nprint(a),0.0


In [5]:
df_unite.dropna(inplace=True)

df_unite.to_csv("../../data/generated/dataset.csv")

In [6]:
list_of_sent = []

for sen in df_unite['solution']:

    tokens = word_tokenize(sen)
    list_of_sent.append(tokens)

In [ ]:
word2vec_model = Word2Vec(
    sentences=list_of_sent,
    vector_size=16,  # Embedding dimension
    window=1,       # Context window size (1 for single words)
    min_count=1,    # Include all words, even rare ones
    workers=4       # Parallelize training
)

# Function to get embedding for a word (fallback to random if OOV)
def get_embedding(word, model):
    try:
        return model.wv[word]
    except KeyError:
        return np.random.rand(model.vector_size)  # Random vector for out-of-vocab words

# Create embedded features
embedded_features = []
for sent in list_of_sent:
    embedded_features.append(np.array([get_embedding(w, word2vec_model) for w in sent]))

# Pad sequences to max length
max_len = max(len(seq) for seq in embedded_features)
padded_features = np.array([
    np.vstack((
        seq,
        np.zeros((max_len - len(seq), word2vec_model.vector_size))  # Pad with zeros
    )) for seq in embedded_features
])

# Flatten each sequence into a 1D vector (for DecisionTree)
X = padded_features.reshape(len(padded_features), -1)  # Shape: (n_samples, max_len * vector_size)
y = df_unite['target'].values

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train decision tree
clf = DecisionTreeRegressor(max_depth=5, random_state=42)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)
print(f"MSE: {mean_squared_error(y_test, y_pred):.2f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.2f}")
print(f"R2: {r2_score(y_test, y_pred):.2f}")


MSE: 0.09
MAE: 0.18
R2: 0.59


In [13]:
prediction = clf.predict(X_test)
idx = 4
print(f"Prediction for new sample: {prediction[idx]}")
print(f"Real: {y_test[idx]}")

Prediction for new sample: 0.9853157121879589
Real: 1.0
